###### 2020-10-26 월요일
###### 2020-10-27 화요일

In [1]:
# 워닝옵션 제거

import warnings
warnings.filterwarnings(action='ignore')

# 01_Machine Learning_(1)
   - Machine Learning(기계학습)이란?
       - 사람의 학습방식을 기계에 적용시킨 방법이다.
       - 수능의 예를 들어 사람의 학습방식을 설명하겠다
       - 수능을 잘 보기위해 정답이 존재하는 수많은 모의고사와 수많은 학습지를 통해 학습하여 수능을 치르는것 처럼, 기계에게도 사람이 학습하는 것 처럼 예측하고자 하는 분야를 학습시키고 예측을 수행하는 방법을 뜻한다 (이해가 될까유..?)

### 목차

#### 1. Machine Learning 체험 (Hold Out Validation)

#### 2. K-Fold Cross Validation

#### 3. Cross_val_score()

#### 4. Hyper Parameter Tuning

## 1. Machine Learning 체험

In [2]:
from sklearn.datasets import load_iris   # iris 데이터셋을 가져오겠다
from sklearn.tree import DecisionTreeClassifier   # DecisionTree의 모듈을 가져오겠다
from sklearn.model_selection import train_test_split    # train / test 를 분리해주는 함수
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# scikit-learn 버전확인
import sklearn
print(sklearn.__version__)

0.23.2


### 간단한 머신러닝을 구현하여 실습해보자
   - DesicionTree를 사용할 것이다.

### (1) 데이터 로딩

In [4]:
iris = load_iris()

 - scikit-learn으로 부터 불러들인 'iris'데이터는 DataFrame형태가 아닌 Bunch형태이다.
 - 그러므로 데이터의 타입을 확인하는 작업이 항상 필요하다.

In [5]:
print(type(iris))

<class 'sklearn.utils.Bunch'>


 - scikit-learn으로 부터 불러들인 Bunch는 key값들을 가지고 있다.
     - data : X 변수들 (설명변수)
     - target : Y 변수 (종속변수)

In [6]:
keys = iris.keys()
print(keys)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


 - iris의 key값들에 접근해보자

In [7]:
print('key data')
display( iris.data[0:10] )
print('*' * 100)
print()

print('key target')
display( iris.target[0:10] )
print('*' * 100)
print()

print('key feature_names')
display( iris.feature_names )
print('*' * 100)
print()

print('key target_names')
display( iris.target_names )
print('*' * 100)
print()

print('key DESCR')
print( iris.DESCR )
print('*' * 100)
print()

key data


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

****************************************************************************************************

key target


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

****************************************************************************************************

key feature_names


['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

****************************************************************************************************

key target_names


array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

****************************************************************************************************

key DESCR
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    

 ##### Feature data set을 확인해보자

In [8]:
iris_data = iris.data
iris_data[0:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

##### label (target) data set을 확인해보자

In [9]:
iris_label = iris.target
iris_label[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

##### 위의 `iris_data`와 `iris_label`을 이용하여 `iris_df` DataFrame으로 만들어보자

In [10]:
iris_df = pd.DataFrame(data=iris_data, index=iris_label)
iris_df.index.name = 'target'
iris_df.columns = iris.feature_names
iris_df.reset_index(inplace=True)

iris_df.head(10)

,target,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,0,4.9,3.0,1.4,0.2
2,0,4.7,3.2,1.3,0.2
3,0,4.6,3.1,1.5,0.2
4,0,5.0,3.6,1.4,0.2
5,0,5.4,3.9,1.7,0.4
6,0,4.6,3.4,1.4,0.3
7,0,5.0,3.4,1.5,0.2
8,0,4.4,2.9,1.4,0.2
9,0,4.9,3.1,1.5,0.1


### (2) train / test 분리

 - train_test_split() 
     - test_size : test set의 비율을 지정하는 옵션
     - random_state : seed를 유지시켜 몇번을 반복하더라도 train / test 가 변하지 않고 유지되게하는 옵션

In [11]:
x_train, x_test, y_train, y_test = train_test_split(iris_data,   # Feature data 입력 
                                                    iris_label,  # Target data 입력
                                                    test_size    = 0.2,  # test set의 비율을 지정하는 옵션
                                                    random_state = 123)  # np.random.seed()와 같은 기능의 옵션

In [12]:
print('train_set')
print('x_train의 길이 :', len(x_train))
print('y_train의 길이 :', len(y_train))
print()

print('test_set')
print('x_test의 길이 :', len(x_test))
print('y_test의 길이 :', len(y_test))
print()

train_set
x_train의 길이 : 120
y_train의 길이 : 120

test_set
x_test의 길이 : 30
y_test의 길이 : 30



### (3) Machine Learning 학습하기
   - DecisionTreeClasiifier라는 머신러닝의 한 종류를 사용하여 위에서 train/test로 나눈 iris데이터를 학습시켜보자

In [13]:
iris_dtc = DecisionTreeClassifier(random_state=20)  # DecisionTreeClassifier() 모델을 불러온다
iris_dtc.fit(x_train, y_train)   # train_set을 사용하여 DecisionTreeClassifier() 모델을 학습시킨다.

DecisionTreeClassifier(random_state=20)

### (4) Machine Learning 예측하기

In [14]:
prediction = iris_dtc.predict(x_test)
print('예측 값 :', prediction)
print('실제 값 :', y_test)

예측 값 : [1 2 2 1 0 1 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0]
실제 값 : [1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0]


### (5) 예측 정확도 평가


In [15]:
from sklearn.metrics import accuracy_score
print('예측 정확도 : ', (accuracy_score(y_test, prediction)) )

# 함수 사용하지 않고 정확도 구현하기
sum( prediction == y_test ) / len(prediction)

예측 정확도 :  0.9666666666666667


0.9666666666666667

## 위에서 만든 `iris_df` 데이터프레임을 사용하여 위 과정을 다시한번 진행해보자!

In [16]:
x_data = iris_df.iloc[ :, 1:]
display(x_data.head(10))

y_data = iris_df['target']
display(y_data[0:10])

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: target, dtype: int64

### (6) train / test 분리하기

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_data,   # Feature data 입력 
                                                    y_data,  # Target data 입력
                                                    test_size    = 0.2,  # test set의 비율을 지정하는 옵션
                                                    random_state = 123)  # np.random.seed()와 같은 기능의 옵션
print('train_set')
print('x_train의 길이 :', len(x_train))
print('y_train의 길이 :', len(y_train))
print()

print('test_set')
print('x_test의 길이 :', len(x_test))
print('y_test의 길이 :', len(y_test))
print()

train_set
x_train의 길이 : 120
y_train의 길이 : 120

test_set
x_test의 길이 : 30
y_test의 길이 : 30



### (7) Machine Learnin 학습 / 예측 / 평가

In [18]:
# 학습
iris_dtc = DecisionTreeClassifier(random_state=20)  # DecisionTreeClassifier() 모델을 불러온다
iris_dtc.fit(x_train, y_train)   # train_set을 사용하여 DecisionTreeClassifier() 모델을 학습시킨다.

# 예측
prediction = iris_dtc.predict(x_test)
print('예측 값 :', prediction)
print('실제 값 :', np.array(y_test))
print()
print()

# 평가
from sklearn.metrics import accuracy_score
print('예측 정확도 : ', (accuracy_score(y_test, prediction)) )

# 함수 사용하지 않고 정확도 구현하기
sum( prediction == y_test ) / len(prediction)

예측 값 : [1 2 2 1 0 1 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0]
실제 값 : [1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0]


예측 정확도 :  0.9666666666666667


0.9666666666666667

## 학습을 시킬때 테스트 데이터를 이용하지 않고 학습데이터 세트로만 학습하고 예측한다면?
   - 학습시킨 데이터를 재사용하여 예측하면, 모의고사를 본 시험지로 재시험을 치르는 것과 같다.
   - 학습시킨 데이터를 재사용하는 것은 말이 되지 않는다.
   - 한번 나쁜 예를 실습해보자
  

In [19]:
from sklearn.datasets import load_iris   # iris 데이터셋을 가져오겠다
from sklearn.tree import DecisionTreeClassifier   # DecisionTree의 모듈을 가져오겠다
from sklearn.model_selection import train_test_split    # train / test 를 분리해주는 함수
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
bad_iris = load_iris()
train_data = iris.data
train_label = iris.target

bad_iris_clf = DecisionTreeClassifier()
bad_iris_clf.fit(train_data, train_label)       # 학습데이터(train_data)와
bad_predict = bad_iris_clf.predict(train_data)  # 예측데이터(train_data)가 같다..
                                                # 절대 일어나면 안되는 일

sum( train_label == bad_predict ) / len(train_label)  # 예측정확도가 1이 나온것을 볼 수있다.
                                                       # 이것은 시험을 치르기전에 
                                                       # 시험지가 유출되서 모든 사람이 정답을 알고 있는것과 같은원리


1.0

## 2. K-Fold Cross Validation

 - `1.` 에서 Data set을 train / test로 나누어서 예측을 시행했던 방식을 **Hold Out Validation** 방식이라고 한다.
 - **Hold Out Validation**은 train / test 로나누기도 하지만, 보통 train / test / validation 세가지로 나누어 진행한다.
 - **K-Fold Cross Validation**은 주어진 데이터 셋을 train / test / validation으로 나눈 **Hold Out**방식을 `K`번 시행하는 것을 말한다.

In [21]:
from sklearn.datasets import load_iris   # iris 데이터셋을 가져오겠다
from sklearn.tree import DecisionTreeClassifier   # DecisionTree의 모듈을 가져오겠다
from sklearn.model_selection import train_test_split, KFold    # train / test 를 분리해주는 함수
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

##### `iris`데이터를 이용하여 5-Fold Cross Validation을 실습해보자

In [22]:
# iris 데이터 불러오기
fold_iris = load_iris()
features = fold_iris.data   # 설명변수(X) 데이터
label = fold_iris.target    # 종속변수(Y) 데이터


# DecisionTreeClassifier() 모델 생성
fold_df_clf = DecisionTreeClassifier()

 ### (1) KFold()
     - n_split=5 : Fold를 5개로 나누는 옵션
     - shuffle = [boolean]
         - True : train / test 인덱스를 무작위로 부여한다
         - False : 데이터프레임의 인덱스 순서를 유지한체로 5등분하여 그 5등분의 data set을 K-fold set으로 사용하는 것이다.
         
##### 말로 설명하기 너무어렵다..
##### shuffle= True / False의 차이점을 이해하기 위해 아래의 결과를 한번 보라

In [23]:
kfold01 = KFold(n_splits=2, shuffle=False)

k = 1
for train_idx, test_idx in kfold01.split(features):
    print(k, '번째 fold')
    print('train_set의 인덱스')
    print(train_idx)
    print()
    print('test_set의 인덱스')
    print(test_idx)
    print()
    print('*'*100)
    
    k += 1
    
    
### fold의 인덱스가 무작위로 선정되지 않음을 볼 수 있다.

1 번째 fold
train_set의 인덱스
[ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149]

test_set의 인덱스
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74]

****************************************************************************************************
2 번째 fold
train_set의 인덱스
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74]

test_set의 인덱스
[ 75  76  77  78  79  80  81  82  83  84  85  86  87

In [24]:
kfold02 = KFold(n_splits=2, shuffle=True)


k = 1
for train_idx, test_idx in kfold02.split(features):
    print(k, '번째 fold')
    print('train_set의 인덱스')
    print(train_idx)
    print()
    print('test_set의 인덱스')
    print(test_idx)
    print()
    print('*'*100)
    
    k += 1
    
    
## fold의 인덱스가 무작위로 선정되었음을 볼 수 있다.

1 번째 fold
train_set의 인덱스
[  2   4   5   6   9  11  13  16  17  19  24  26  31  32  35  36  37  40
  41  42  43  44  46  47  55  56  57  59  60  61  67  69  70  73  74  75
  76  79  80  81  83  86  87  88  91  92  93  95  96  97  99 101 102 104
 105 108 110 111 113 117 120 121 122 123 124 127 129 132 134 136 138 142
 144 148 149]

test_set의 인덱스
[  0   1   3   7   8  10  12  14  15  18  20  21  22  23  25  27  28  29
  30  33  34  38  39  45  48  49  50  51  52  53  54  58  62  63  64  65
  66  68  71  72  77  78  82  84  85  89  90  94  98 100 103 106 107 109
 112 114 115 116 118 119 125 126 128 130 131 133 135 137 139 140 141 143
 145 146 147]

****************************************************************************************************
2 번째 fold
train_set의 인덱스
[  0   1   3   7   8  10  12  14  15  18  20  21  22  23  25  27  28  29
  30  33  34  38  39  45  48  49  50  51  52  53  54  58  62  63  64  65
  66  68  71  72  77  78  82  84  85  89  90  94  98 100 103 106 107 109
 1

### 이제 iris 데에터에 KFold를 본격적으로 적용해보자
   - shuffle = True로하여 Train와 Test의 인덱스를 무작위로 부여할 것이다.

In [25]:
# Kfold를 나눌 인덱스를 생성
kfold = KFold(n_splits=5, shuffle=True)

# feature data
print('iris shape', features.shape[0])

iris shape 150


In [26]:
# 각각의 K-fold set으로 부터 나오는 정확도를 담을 리스트
cv_accuracy = []

k = 1
for train_idx, test_idx in kfold.split(features): # features라는 데이터의 K-fold를 시행하기위해 train, test 인덱스를 생성하겠다!
    x_train = features[train_idx]
    y_train = label[train_idx]
    
    x_test = features[test_idx]
    y_test = label[test_idx]
    
    
    # K번째 fold의 학습을 진행
    fold_df_clf.fit(x_train, y_train)
    
    # K번째 fold의 예측을 진행
    y_pred = fold_df_clf.predict(x_test)
    
    # k번째 fold의 정확도를 기록
    accuracy = accuracy_score(y_test, y_pred)
    cv_accuracy.append(accuracy)
    
    print(k, '번째 Fold')
    print(k, '번째 Fold의 real y :', y_test)
    print(k, '번째 Fold의 pred y :', y_pred)
    print(k, '번째 Fold의 accuracy :', accuracy)
    print('*'*100)
    print()
    k += 1

1 번째 Fold
1 번째 Fold의 real y : [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
1 번째 Fold의 pred y : [0 0 0 0 0 0 0 1 1 1 1 2 1 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
1 번째 Fold의 accuracy : 0.9333333333333333
****************************************************************************************************

2 번째 Fold
2 번째 Fold의 real y : [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
2 번째 Fold의 pred y : [0 0 0 0 0 0 0 0 0 0 0 0 1 2 1 1 1 1 1 1 2 1 2 2 2 2 2 2 2 2]
2 번째 Fold의 accuracy : 0.9333333333333333
****************************************************************************************************

3 번째 Fold
3 번째 Fold의 real y : [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
3 번째 Fold의 pred y : [0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 1 2 2 1 2 2 2 2]
3 번째 Fold의 accuracy : 0.9333333333333333
****************************************************************************************************

4 번째 Fold
4 번째 Fold의 real y : [0 0 0 0 0 0 0 0 0 

##### 5-Fold Cross Validation 결과

In [27]:
print('5-Fold Cross Validation의 평균결과 : ', np.mean(cv_accuracy))

5-Fold Cross Validation의 평균결과 :  0.9466666666666667


### (2) Stratified K Fold
   - KFold() 함수는 shuffle=True이면 Target이 균등하든 안하든, 무작위로 인덱스를 선정하여 Train / Test를 나누고 shuffle=False이면, 일정한 규칙에 따라 무작위로 인덱스를 선정하여 Train / Test를 나누는 방법이다.
   - Straified KFold() 함수는 KFold() 함수와는 다르게 레이블의 분포를 먼저 고려하여 train / test 인덱스를 부여하는 방법이다
  

### 여기서 잠깐..!
##### 기존 KFold()함수의 단점을 짚고 넘어가자

In [28]:
kfold_iris_data = load_iris()

kfold_iris_df = pd.DataFrame(data    =  kfold_iris_data.data, 
                             columns =  kfold_iris_data.feature_names)

kfold_iris_df['target'] = kfold_iris_data.target


In [29]:
kfold_iris = KFold(n_splits=3)
k = 1

for train_idx, test_idx in kfold_iris.split(kfold_iris_df): # features라는 데이터의 K-fold를 시행하기위해 train, test 인덱스를 생성하겠다!
    x_train = kfold_iris_df.loc[train_idx, : ].drop(['target'], axis=1, inplace=False)
    y_train = kfold_iris_df.loc[train_idx,  'target']
    
    x_test = kfold_iris_df.loc[test_idx, : ].drop(['target'], axis=1, inplace=False)
    y_test = kfold_iris_df.loc[test_idx,  'target']
    
    
    # K번째 fold의 학습을 진행
    fold_df_clf.fit(np.array(x_train), np.array(y_train))
    
    # K번째 fold의 예측을 진행
    y_pred = fold_df_clf.predict(np.array(x_test))
    
    # k번째 fold의 정확도를 기록
    accuracy = accuracy_score(y_test, y_pred)
    cv_accuracy.append(accuracy)
    
    print(k, '번째 Fold')
    print(k, '번째 Fold의 y_train target :\n', y_train.value_counts())
    print()
    print(k, '번째 Fold의 y_test target :\n', y_test.value_counts())
    print()
    print(k, '번째 Fold의 accuracy :', accuracy)
    print('*'*100)
    print()
    k += 1

1 번째 Fold
1 번째 Fold의 y_train target :
 2    50
1    50
Name: target, dtype: int64

1 번째 Fold의 y_test target :
 0    50
Name: target, dtype: int64

1 번째 Fold의 accuracy : 0.0
****************************************************************************************************

2 번째 Fold
2 번째 Fold의 y_train target :
 2    50
0    50
Name: target, dtype: int64

2 번째 Fold의 y_test target :
 1    50
Name: target, dtype: int64

2 번째 Fold의 accuracy : 0.0
****************************************************************************************************

3 번째 Fold
3 번째 Fold의 y_train target :
 1    50
0    50
Name: target, dtype: int64

3 번째 Fold의 y_test target :
 2    50
Name: target, dtype: int64

3 번째 Fold의 accuracy : 0.0
****************************************************************************************************



##### 위 결과를 잘 보면, 모든 Fold에서 정확도가 0이다..!
   - 1-Fold의 경우
       - Train set의 target값은 1과 2만을 가진다.
       - Test set의 target값은 0의 값을 가진다
           ##### 이 경우 test set의 target값인 0을 할습할 수 없기 때문에 정확도가 0이 나온다!
   - 2-Fold의 경우 / 3-Fold의 경우 
       - **Test set의 target값을 가지고 있지않아 학습을 할 수없기 때문에, 정확도가 0으로 나타나게 된다!!**
       
       
       
### 그래서 이러한 문제점을 방지하기 위해...
   - **Stratified K Fold**를 사용하여 위의 문제점을 방지한다.

In [30]:
# 레이블 값의 분포를 반영해주지 못하는 문제를 해결하기위해서
from sklearn.model_selection import StratifiedKFold

skf_iris = StratifiedKFold(n_splits=3)
k = 1

for train_idx, test_idx in skf_iris.split(kfold_iris_df, kfold_iris_df['target']):# 이부분에서 label값이 들어간다는게 KFold()함수와의 차이점
    x_train = kfold_iris_df.loc[train_idx, : ].drop(['target'], axis=1, inplace=False)
    y_train = kfold_iris_df.loc[train_idx,  'target']
    
    x_test = kfold_iris_df.loc[test_idx, : ].drop(['target'], axis=1, inplace=False)
    y_test = kfold_iris_df.loc[test_idx,  'target']
    
    
    # K번째 fold의 학습을 진행
    fold_df_clf.fit(np.array(x_train), np.array(y_train))
    
    # K번째 fold의 예측을 진행
    y_pred = fold_df_clf.predict(np.array(x_test))
    
    # k번째 fold의 정확도를 기록
    accuracy = accuracy_score(y_test, y_pred)
    cv_accuracy.append(accuracy)
    
    print(k, '번째 Fold')
    print(k, '번째 Fold의 y_train target :\n', y_train.value_counts())
    print()
    print(k, '번째 Fold의 y_test target :\n', y_test.value_counts())
    print()
    print(k, '번째 Fold의 accuracy :', accuracy)
    print('*'*100)
    print()
    k += 1

1 번째 Fold
1 번째 Fold의 y_train target :
 2    34
1    33
0    33
Name: target, dtype: int64

1 번째 Fold의 y_test target :
 1    17
0    17
2    16
Name: target, dtype: int64

1 번째 Fold의 accuracy : 0.98
****************************************************************************************************

2 번째 Fold
2 번째 Fold의 y_train target :
 1    34
2    33
0    33
Name: target, dtype: int64

2 번째 Fold의 y_test target :
 2    17
0    17
1    16
Name: target, dtype: int64

2 번째 Fold의 accuracy : 0.92
****************************************************************************************************

3 번째 Fold
3 번째 Fold의 y_train target :
 0    34
2    33
1    33
Name: target, dtype: int64

3 번째 Fold의 y_test target :
 2    17
1    17
0    16
Name: target, dtype: int64

3 번째 Fold의 accuracy : 1.0
****************************************************************************************************



##### Stratified K Fold는 y_train과 y_test의 target값[0, 1, 2] 이 골고루 들어가 있음을 볼 수 있다!

### 참고로... `Stratified K Fold`는 분류(Classification)에서만 적용할 수 있다.
### 회귀에서는 적용할 수 없다.


### 실습
   - iris 데이터에서 Stratified KFold를 적용하여 3Fold 교차검증을 진행하고, 평균 정확도를 확인하라
   - `random_state = 100`으로 세팅한다

In [31]:
# 데이터 불러오기
iris_data = load_iris()
x_data = iris_data.data
y_data = iris_data.target

# 데이터프레임만들기
iris_df = pd.DataFrame(data=x_data, columns=iris_data.feature_names)
iris_df['y'] = y_data

iris_df.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


In [32]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# stratifiedKFold 불러오기
kfold = StratifiedKFold(n_splits=3, random_state=100)

# DecisionTreeClassifier() 모델 생성
model = DecisionTreeClassifier(random_state=100)

# 각각의 fold의 accuracy를 담는 리스트
K_accuracy = []
K = 1

for train_idx, test_idx in kfold.split(iris_df, iris_df['y']):
    x_train, y_train = iris_df.loc[train_idx, :].drop(['y'], axis=1, inplace=False), iris_df.loc[train_idx, 'y']
    x_test, y_test = iris_df.loc[test_idx, :].drop(['y'], axis=1, inplace=False), iris_df.loc[test_idx, 'y']
    
    model.fit(np.array(x_train), np.array(y_train))
    y_pred = model.predict(np.array(x_test))
    
    accuracy = accuracy_score(y_test, y_pred)
    K_accuracy.append(accuracy)
    
    print(K, '번째 Fold 예측값 : \n', np.array(y_pred))
    print(K, '번째 Fold 실제값 : \n', np.array(y_test))
    print(K, '번째 Fold 정확도 : \n', accuracy)
    print('*'* 100)
    print()
    K += 1


print()
print()
print('전체정확도 :', K_accuracy)
print('3-fold cv의 평균정확도 :', np.mean(K_accuracy))

1 번째 Fold 예측값 : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2]
1 번째 Fold 실제값 : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
1 번째 Fold 정확도 : 
 0.98
****************************************************************************************************

2 번째 Fold 예측값 : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 2 2 2 1
 2 2 2 2 2 2 2 2 2 1 2 2 2]
2 번째 Fold 실제값 : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
2 번째 Fold 정확도 : 
 0.92
****************************************************************************************************

3 번째 Fold 예측값 : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 1 2 2 2 2 2 2 2 2 2 2 2]
3 번째 Fold 실제값 : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
3 번째 Fold 정확도 : 
 0.96
*

## 3. cross_val_score() / cross_validate()
   - 지금까지 k-Fold cv를 loop를 구현하여 실습해보았다.
   - 지금부터는 함수를 적용하여 K-fold cv를 실습해보자.
   - cross_val_score(`머신러닝`, `Feature`, `Y`, `(F1 score와 같은 평가지표)`, `cv`)
   - cross_val_score()는 Cross Validation을 수행하기 위해 KFold()를 사용하는 것이 아니라 StratifiedKFold()방식을 사용한다.
   - cross_val_score()과 cross_validate()의 차이점은 각각의 Fold가 처리된 수행시간을 출력하느냐 아니냐의 차이이다.

In [33]:
from sklearn.datasets import load_iris   # iris 데이터셋을 가져오겠다
from sklearn.tree import DecisionTreeClassifier   # DecisionTree의 모듈을 가져오겠다
from sklearn.model_selection import cross_val_score, cross_validate  # 지금하는 함수!
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

### cross_val_score()

In [34]:
# 데이터불러오기
csv_iris = load_iris()
csv_iris_feature =  csv_iris.data
csv_iris_label   =  csv_iris.target

# 머신러닝 가져오기
csv_iris_dtc = DecisionTreeClassifier(random_state=200)

# cross_val_score() 함수 사용하여 K-fold cv 수행하기
scoring = cross_val_score(csv_iris_dtc, csv_iris_feature, csv_iris_label, scoring='accuracy', cv=3)
# scoring = cross_validate(csv_iris_dtc, csv_iris_feature, csv_iris_label, scoring='accuracy', cv=3)  < 나중에 이걸로도 해보세요
scoring

array([0.98, 0.92, 1.  ])

### cross_validate()

In [35]:
# 데이터불러오기
csv_iris = load_iris()
csv_iris_feature =  csv_iris.data
csv_iris_label   =  csv_iris.target

# 머신러닝 가져오기
csv_iris_dtc = DecisionTreeClassifier(random_state=200)

# cross_validate() 함수 사용하여 K-fold cv 수행하기
scoring = cross_validate(csv_iris_dtc, csv_iris_feature, csv_iris_label, scoring='accuracy', cv=3)
scoring

{'fit_time': array([0.00099802, 0.00199389, 0.        ]),
 'score_time': array([0.        , 0.        , 0.00099707]),
 'test_score': array([0.98, 0.92, 1.  ])}

## 4. Hyper Parameter Tuning
   - 머신러닝의 내부 알고리즘을 일일이 설명할 수 없지만, 머신러닝 내부에는 알고리즘적으로 수치를 필수적으로 동반한다.
   - 이러한 수치를 변화시켜 데이터의 학습정도나, 학습방법을 사용자가 직접 설정할 수 있다.
   - 하지만, 데이터마다 최적의 Hyper Parameter가 모두 다르기 때문에, 최적의 예측을 수행하기 위해서는 Hyper Parameter를 찾는 작업이 데이터분석 과정에서 필연적으로 동반된다.
   - Hyper Parameter를 찾는 방법중에 대표적인 방법은 **GridSearch**이다.
   - **GridSearch**는 Hyper Parameter값들에 넣고자 하는 값들을 여러가지 입력하면, 차례대로 입력 값을 입력하여 예측을 수행하고 성능이 가장 좋은 Hyper Parameter를 선택하는 방법이다
   - 파이썬은 **GridSearch**와 **CrossValidation**을 함께 수행해주는 함수가 존재한다
   
   
   - `GridSearchCV()`

##### Decision Tree Parameter
   > 의사결정나무의 Hyper Parameter를 간단히 알아보자
   - paramater critetion : 노드를 분리하는 기준(gini계수, entropy)
   - parameter splitter : 노드를 분리하는 방법(random, best)
   - parameter max_depth : 트리모형의 깊이를의미
   - parameter min_samples_split : 브랜치 노드에서 분리가 일어나기 위한 샘플 수
   - parameter min_samples_leaf : 노드에 필요한 최소한의 샘플 수
   - parameter max_features : 노드를 분리할 때, 고려하는 속성의 수

### GridSearchCV()
   - 백번 이론보다 한번 실습이 더 이해가 빠르다!

In [36]:
from sklearn.datasets import load_iris   # iris 데이터셋을 가져오겠다
from sklearn.tree import DecisionTreeClassifier   # DecisionTree의 모듈을 가져오겠다
from sklearn.model_selection import GridSearchCV, train_test_split 
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
# iris 데이터 불러오기
gscv_iris = load_iris()
x_train, x_test, y_train, y_test = train_test_split(gscv_iris.data, 
                                                    gscv_iris.target,
                                                    test_size = 0.2,
                                                    random_state = 120)

print('x_train 길이 :', len(x_train))
print('y_train 길이 :', len(y_train))
print('x_test 길이 :', len(x_test))
print('y_test 길이 :', len(y_test))



# 트리 모델 불러오기
gscv_tree = DecisionTreeClassifier()

x_train 길이 : 120
y_train 길이 : 120
x_test 길이 : 30
y_test 길이 : 30


In [38]:
# 찾고샂하는 Hyper Parameter 지정하기
params = {
    'criterion'         : ['gini', 'entropy'],
    'splitter'          : ['random', 'best'],
    'max_depth'         : [1, 2, 3],
    'min_samples_split' : [2, 3]
}



# GridSeachCV사용하기 : Hyper Parameter 찾고, Cross Validation 수행하기위한 준비
grid_gscv_tree = GridSearchCV(gscv_tree, 
                              param_grid = params, 
                              cv         = 3, 
                              refit      = True)  # 최적의 Hyper Parameter를 찾고 재학습을 하겠다는 의미



In [39]:
# 학습 수행 : Grid Search / Cross Validation 수행
grid_gscv_tree.fit(x_train, y_train)

# 학습 결과 출력
grid_gscv_tree.cv_results_

{'mean_fit_time': array([0.00099699, 0.00066487, 0.0006632 , 0.00066431, 0.00066503,
        0.00033251, 0.00033228, 0.00066447, 0.00066487, 0.00033299,
        0.00100255, 0.00099929, 0.00066574, 0.        , 0.00100851,
        0.00099754, 0.00066638, 0.00099683, 0.00100764, 0.00033212,
        0.00066614, 0.0006756 , 0.00065335, 0.        ]),
 'std_fit_time': array([4.89903609e-07, 4.70134691e-04, 4.68955642e-04, 4.69740766e-04,
        4.70246478e-04, 4.70246438e-04, 4.69909263e-04, 4.69853319e-04,
        4.70134086e-04, 4.70920787e-04, 8.21415985e-04, 4.51249109e-06,
        4.70752291e-04, 0.00000000e+00, 1.58484107e-05, 5.15042996e-07,
        4.71204672e-04, 7.01885292e-07, 1.71620900e-05, 4.69684480e-04,
        4.71033340e-04, 4.77901159e-04, 4.62195281e-04, 0.00000000e+00]),
 'mean_score_time': array([0.00033243, 0.00033267, 0.00033236, 0.0003341 , 0.00033251,
        0.00033251, 0.00033275, 0.00033124, 0.00033267, 0.00066447,
        0.00065955, 0.00067329, 0.        , 0.00

##### 근데 위 결과 보기가좀.. 힘들지 않음?
##### 쉽게보게 해줄게여

In [40]:
score_df = pd.DataFrame(grid_gscv_tree.cv_results_)
display(score_df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000997,4.899036e-07,0.000332,0.000470,gini,1,2,random,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.650,0.650,0.65,0.650000,0.000000,24
1,0.000665,4.701347e-04,0.000333,0.000470,gini,1,2,best,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.675,0.675,0.65,0.666667,0.011785,17
2,0.000663,4.689556e-04,0.000332,0.000470,gini,1,3,random,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.675,0.675,0.65,0.666667,0.011785,17
3,0.000664,4.697408e-04,0.000334,0.000472,gini,1,3,best,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.675,0.675,0.65,0.666667,0.011785,17
4,0.000665,4.702465e-04,0.000333,0.000470,gini,2,2,random,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.875,0.925,0.85,0.883333,0.031180,11


In [41]:
# 성능이 가장 좋은 결과부터 출력
score_df[ ['params', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score'] ].sort_values(by='rank_test_score')

,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score
23,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",1,0.950,0.900,0.975
21,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",1,0.950,0.900,0.975
22,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",3,0.950,0.975,0.875
9,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",3,0.950,0.875,0.975
11,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",3,0.950,0.875,0.975
5,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",6,0.950,0.900,0.875
7,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",6,0.950,0.900,0.875
19,"{'criterion': 'entropy', 'max_depth': 2, 'min_...",6,0.950,0.900,0.875
17,"{'criterion': 'entropy', 'max_depth': 2, 'min_...",6,0.950,0.900,0.875
10,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",10,0.975,0.825,0.875


##### 더 간단히 최적의 경우만 보고싶다면?

In [42]:
print('최적의 하이퍼 파라미터는 ?', grid_gscv_tree.best_params_)
print('최적의 하이퍼 파라미터일 때의 정확도는 ?', grid_gscv_tree.best_score_)

최적의 하이퍼 파라미터는 ? {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2, 'splitter': 'best'}
최적의 하이퍼 파라미터일 때의 정확도는 ? 0.9416666666666668


### 최적의 하이퍼파라매터를 이용하여 최종 test set 예측하기

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
estimator = grid_gscv_tree.best_estimator_
y_pred = estimator.predict(x_test)

print('test set의 실제 y 값', y_test)
print('test set의 예측 y 값', y_pred)
print('최종 Test set 예측에 대한 정확도는?', accuracy_score(y_test, y_pred))

test set의 실제 y 값 [1 2 1 1 2 0 2 1 1 2 0 1 0 1 1 0 0 0 0 0 2 0 2 0 1 2 1 0 0 0]
test set의 예측 y 값 [1 2 1 1 2 0 2 1 1 2 0 1 0 1 1 0 0 0 0 0 2 0 2 0 1 2 1 0 0 0]
최종 Test set 예측에 대한 정확도는? 1.0


### [실습]
   1. `breast_cancer`데이터를 불러오고 데이터프레임으로 만들어라
   2. train과 test로 나누어라(7:3)
   3. 학습과 예측을 수행해보아라
   4. 학습 결과에 대한 평가도 해보아라
   5. 교차검을을 수행하라(cv)
   6. GridSearch를 동반한 교차검증을 수행하라


In [45]:
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split 
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

##### 1. breast_cancer데이터를 불러오고 데이터프레임으로 만들어라

In [46]:
cancer = load_breast_cancer()
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [47]:
cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [48]:
cancer_df = pd.DataFrame(data    = cancer.data,
                         columns = cancer.feature_names)
cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

##### 2. train과 test로 나누어라(7:3)

In [49]:
x_train, x_test, y_train, y_test = train_test_split(np.array(cancer_df), 
                                                    cancer.target, 
                                                    test_size = 0.3)

print('x_train 크기 :', len(x_train))
print('y_train 크기 :', len(y_train))
print('x_test 크기 :', len(x_test))
print('y_test 크기 :', len(y_test))

x_train 크기 : 398
y_train 크기 : 398
x_test 크기 : 171
y_test 크기 : 171


##### 3. 학습과 예측을 수행해보아라

In [50]:
# 모델 불러오기
decisionTree = DecisionTreeClassifier(random_state=123)

# 학습
decisionTree.fit(x_train, y_train)

# 예측
y_pred = decisionTree.predict(x_test)

##### 4. 학습 결과에 대한 평가도 해보아라

In [51]:
print('train과 test 비율을 7:3으로 시행한 결과의 정확도는?',  accuracy_score(y_test, y_pred))

train과 test 비율을 7:3으로 시행한 결과의 정확도는? 0.9239766081871345


##### 5. 교차검을을 수행하라(cv)

In [52]:
# 모델 불러오기
decisionTree = DecisionTreeClassifier(random_state=123)

# KFold를 수행하기위해 StratifiedKFold방식을 이용해 인덱스 부여
k = StratifiedKFold(n_splits=5)

# 5-Fold Cross Validation 수행
k_fold_accuracy = []
cnt = 1
for train_idx, test_idx in k.split(cancer.data, cancer.target):
    x_train, x_test = cancer.data[train_idx], cancer.data[test_idx]
    y_train, y_test = cancer.target[train_idx], cancer.target[test_idx]
    
    decisionTree.fit(x_train, y_train)
    y_pred = decisionTree.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    k_fold_accuracy.append(accuracy)
    
    print(cnt,'번째 Fold의 accuracy는?', accuracy)
    cnt += 1

print()
print('5-Fold Cross Validation의 평균 accuracy는?', np.mean(k_fold_accuracy))


1 번째 Fold의 accuracy는? 0.9122807017543859
2 번째 Fold의 accuracy는? 0.9298245614035088
3 번째 Fold의 accuracy는? 0.9298245614035088
4 번째 Fold의 accuracy는? 0.9122807017543859
5 번째 Fold의 accuracy는? 0.8938053097345132

5-Fold Cross Validation의 평균 accuracy는? 0.9156031672100605


##### 6. GridSearch를 동반한 교차검증을 수행하라 

In [53]:
# 데이터 나누기
x_train, x_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2)

In [56]:
# 모델 불러오기
decisionTree = DecisionTreeClassifier(random_state=123)

# GridSearch를 수행할 Parameter 지정하기
params = {
    'criterion'         : ['gini', 'entropy'],
    'splitter'          : ['random', 'best'],
    'max_depth'         : [6, 8, 10],
    'min_samples_split' : [3, 4, 5]
}


# GridSearchCV함수 준비하기

grid_cv_decision = GridSearchCV(decisionTree, 
                                param_grid = params,
                                cv         = 5,
                                refit      = True)


# GridSearchCV함수 실행하기
grid_cv_decision.fit(x_train, y_train)


# 결과 데이터프레임에 저장하기
temp_df = pd.DataFrame(grid_cv_decision.cv_results_)
result_df = temp_df[ ['params', 'rank_test_score', 'mean_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score'] ]
display(result_df.sort_values(by='rank_test_score').head(10))


# 간단히 GridSearchCV 결과 출력하기
print('최적의 하이퍼 파라매터는?', grid_cv_decision.best_params_)
print('최적의 하이퍼 파라매터의 정확도 결과는?', grid_cv_decision.best_score_)


,params,rank_test_score,mean_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
28,"{'criterion': 'entropy', 'max_depth': 8, 'min_...",1,0.951648,0.945055,0.923077,0.978022,0.967033,0.945055
34,"{'criterion': 'entropy', 'max_depth': 10, 'min...",2,0.949451,0.945055,0.923077,0.978022,0.956044,0.945055
18,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",3,0.940659,0.945055,0.923077,0.923077,0.956044,0.956044
22,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",4,0.938462,0.945055,0.923077,0.923077,0.956044,0.945055
20,"{'criterion': 'entropy', 'max_depth': 6, 'min_...",4,0.938462,0.945055,0.923077,0.923077,0.956044,0.945055
0,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",6,0.936264,0.934066,0.945055,0.923077,0.945055,0.934066
30,"{'criterion': 'entropy', 'max_depth': 10, 'min...",7,0.934066,0.934066,0.923077,0.912088,0.923077,0.978022
16,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",8,0.931868,0.923077,0.901099,0.912088,0.945055,0.978022
12,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",8,0.931868,0.934066,0.912088,0.912088,0.934066,0.967033
32,"{'criterion': 'entropy', 'max_depth': 10, 'min...",10,0.929670,0.934066,0.923077,0.912088,0.923077,0.956044


최적의 하이퍼 파라매터는? {'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 5, 'splitter': 'random'}
최적의 하이퍼 파라매터의 정확도 결과는? 0.9516483516483516


##### 복습
   - `np.hstack`
   - `np.vstack`

In [87]:
a = np.array( [[1, 2], [3, 4]] )
b = np.array( [[5, 6], [7, 8]] )

c = np.hstack([ a, b ])
d = np.vstack([ a, b ])

print(c)
print(d)

[[1 2 5 6]
 [3 4 7 8]]
[[1 2]
 [3 4]
 [5 6]
 [7 8]]


## 5. StandardScaler & 불순도검사
   - 데이터 셋마다 다르지만, 정규화를 수행하여 모델의 성능이 높아지는 경우가 있다.
   - 우리는 `평균과 표준편차를 이용한 정규화 : StandardScaler()`를 실습해보자

In [57]:
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split 
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
criterion_iris = load_iris()

In [63]:
x_train, x_test, y_train, y_test = train_test_split(criterion_iris.data, criterion_iris.target, test_size=0.3, random_state=0)

In [89]:
from sklearn.preprocessing import StandardScaler

# x_train 평균, 표준편차를 구하기 위해서 표준화
sc = StandardScaler()
sc.fit(x_train)   ## x_train의 평균, 표준편차를 기준으로 표준화를 시행하기위한 준비단계

x_train_std = sc.transform(x_train)  ## x_train 표준화
x_test_std = sc.transform(x_test)    ## x_test 표준화

In [90]:
decisionTree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

decisionTree.fit(x_train_std, y_train)
y_pred = decisionTree.predict(x_test_std)

print('총 테스트 개수 : %d, 오류개수 : %d' %( len(y_test), sum(y_test != y_pred) ))
print('정확도 : %.2f' %accuracy_score(y_test, y_pred))

총 테스트 개수 : 45, 오류개수 : 1
정확도 : 0.98


##### 불순도검사

In [107]:
# 불순도plot

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # 마커와 컬러맵을 설정합니다.
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 결정 경계를 그립니다.
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # 테스트 샘플을 부각하여 그립니다.
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    facecolors='none',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

In [116]:
x_combind_std = np.vstack((x_train_std, x_test_std))
# print(x_combind_std)
y_combind = np.hstack((y_train, y_test))
# print(y_combind)
plot_decision_regions(X=x_combind_std , y=y_combind, classifier=decisionTree, test_idx=range(105,150))

ValueError: Number of features of the model must match the input. Model n_features is 4 and input n_features is 2 